<a href="https://colab.research.google.com/github/JakeOh/202505_BD50/blob/main/lab_da/ml05_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

머신 러닝(Machine Learning) 종류 - 사람의 감독 하에 훈련하는 것인 지 아닌 지 분류
*   지도 학습(Supervised Learning): 레이블(타겟)이 있는 데이터를 학습.
    *   분류(classification): 클래스들을 분류.
    *   회귀(regression): 숫자 예측.
*   비지도 학습(Unsupervised Learning): 레이블이 없는 데이터를 학습.
    *   군집(clustering)
    *   시각화(visualization)
    *   이상치(outlier) 탐지
*   준지도 학습(semi-supervised learning): 비지도 학습 + 전문가 시스템
*   강화 학습(reinforcement learning): 보상과 벌칙을 통해서 학습.